# <center> Данный скрипт осуществляет выгрузку данных с комментариями и пользователями,  для их дальнейшего анализа

In [1]:
# Для выгрузки данных с ВК
import requests
import json
import os
from auth_data import token # сервисный токен
from auth_data import tokenbgd # личный токен

# Для обработки полученных данных
import pandas as pd
import numpy as np

# <center> Получаем данные постов группы

In [2]:
# Запрашиваемые параметры
group_name = [# Название группы взятое из ссылки
    'atheism',
    'varlamov',
    'sandy_mustache_vk',
    'rocknrofl',
    'agnpage'    
]
count_last_posts = 40 # Число последних постов

# Создаем список с id номерами постов группы
group_json = list()
post_id_dict = dict()

for j in range(len(group_name)):
    # Производим запрос
    url_group = f"https://api.vk.com/method/wall.get?domain={group_name[j]}&count={count_last_posts}&access_token={token}&v=5.81"
    req = requests.get(url_group)
    group_json.append(req.json())
    post_id_list = list()
    for i in range(len(group_json[j]['response']['items'])):
        post_id_list.append(group_json[j]['response']['items'][i]['id'])
        # Проверяем существует ли директория с именем группы
    if os.path.exists(f"groups/{group_name[j]}"):
        print(f"Директория с именем {group_name[j]} уже существует!")
    else:
        os.makedirs(f"groups/{group_name[j]}")
        # сохраняем данные в json файл, чтобы видеть структуру
    post_id_dict.update({group_name[j]:post_id_list})
        
    with open(f"groups/{group_name[j]}/{group_name[j]}.json", "w", encoding="utf-8") as file:
        json.dump(group_json, file, indent=4, ensure_ascii=False)

print("id постов:",post_id_dict)

Директория с именем atheism уже существует!
Директория с именем varlamov уже существует!
Директория с именем sandy_mustache_vk уже существует!
Директория с именем rocknrofl уже существует!
Директория с именем agnpage уже существует!
id постов: {'atheism': [2795162, 2794790, 2794407, 2794126, 2793803, 2793651, 2793616, 2793544, 2793477, 2793427, 2793321, 2793191, 2793003, 2792911, 2792632, 2792491, 2792311, 2792101, 2792039, 2791899, 2791772, 2791621, 2791384, 2791265, 2791185, 2791046, 2790974, 2790902, 2790861, 2790760, 2790678, 2790636, 2790522, 2790355, 2790229, 2790129, 2790016, 2789793, 2789663, 2789515], 'varlamov': [2363697, 2364719, 2364654, 2364500, 2363995, 2363976, 2363946, 2363885, 2363863, 2363826, 2363778, 2363738, 2363675, 2363660, 2363587, 2363537, 2363456, 2363396, 2363380, 2363330, 2363286, 2363268, 2363264, 2363208, 2363191, 2363147, 2363131, 2363115, 2363081, 2363048, 2363027, 2362908, 2362794, 2362739, 2362734, 2362648, 2362610, 2362582, 2362533, 2362436], 'sandy_m

# <center> Получение комментариев

In [3]:
# Создаем списки с пользователями и текстом комментариев
groups_id = list()
text = list()
post_id_json = list()

for i in range(len(group_name)):
    groups_id.append(group_json[i]['response']['items'][0]['from_id'])

for m in range(len(groups_id)):
    for j in range(len(post_id_dict[group_name[m]])):
        # Производим запрос
        url_posts = f"https://api.vk.com/method/wall.getComments?owner_id={groups_id[m]}&post_id={post_id_dict[group_name[m]][j]}&count=100&access_token={token}&v=5.81"
        req = requests.get(url_posts)
        post_id_json.append(req.json())
        for i in range(len(post_id_json[j]['response']['items'])):
            text.append([
                groups_id[m],                                               # id группы
                post_id_dict[group_name[m]][j],                                        # id поста
                post_id_json[j]['response']['items'][i]['from_id'],     # id пользователя 
                post_id_json[j]['response']['items'][i]['text']],       # текст комментария            
            )

## Проверяем существует ли директория с именем группы
#if os.path.exists(f"groups/{group_name}/{groups_id}/{groups_id}_text")==False:
#    os.makedirs(f"groups/{group_name}/{groups_id}/{groups_id}_text")

## Сохраняем данные в json файл, чтобы видеть структуру       
#with open(f"groups/{group_name}/{groups_id}/{groups_id}_text/{groups_id}_text.json", "w", encoding="utf-8") as file:
#    json.dump(text, file, indent=4, ensure_ascii=False)


# <center> Создаем таблицу, куда будем вносить данные о пользователях

In [4]:
users_data = pd.DataFrame(
    data = text,
    columns= ['id_группы', 'id_поста', 'id_пользователя', 'Комментарий']
)

In [5]:
users_data

,id_группы,id_поста,id_пользователя,Комментарий
0,-63683472,2795162,655386741,Земля стекловатой. Пидар.
1,-63683472,2795162,282596539,Земля пухом😔
2,-63683472,2795162,69696727,"[id655386741|Квантовый], Товарищи! не стесняйт..."
3,-63683472,2795162,241770145,Одним бандеровцем меньше.
4,-63683472,2795162,426871262,Как же пофуй!
...,...,...,...,...
14055,-93574769,1193,23966451,"[id369621923|Пересвет], что не так с моим знан..."
14056,-93574769,1193,69885373,
14057,-93574769,1193,717453163,С Украины берут пример.
14058,-93574769,1193,187193568,


In [6]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14060 entries, 0 to 14059
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_группы        14060 non-null  int64 
 1   id_поста         14060 non-null  int64 
 2   id_пользователя  14060 non-null  int64 
 3   Комментарий      14060 non-null  object
dtypes: int64(3), object(1)
memory usage: 439.5+ KB


# <center> Добавление данных о городе пользователя

In [92]:
# Напишем функцию, возвращающую имя пользователя
def user_info(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city,bdate,is_closed&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        first_name  = users_info_json['response'][0]['first_name']
        last_name   = users_info_json['response'][0]['last_name']
        birthday    = users_info_json['response'][0]['bdate']
        #zakritost   = users_info_json['response'][0]['is_closed']
        city        = users_info_json['response'][0]['city']['title']         
    except KeyError:
        city = 0
    except UnboundLocalError:
        return [first_name, last_name, 0, city]
    except IndexError:
        return 0, 0, 0, 0

    #return users_info_json
    return [first_name, last_name, birthday, city]
    #return [first_name, last_name, birthday, zakritost, city]

In [81]:
print(user_info(655386745))
print(user_info(655386744))

{'response': [{'id': 655386745, 'deactivated': 'banned', 'first_name': 'Grace', 'last_name': 'Thompson', 'can_access_closed': True, 'is_closed': False}]}
{'response': [{'id': 655386744, 'bdate': '7.7.1997', 'first_name': 'Raimundo', 'last_name': 'Freytas'}]}


In [95]:
print(user_info(655386745))
print(user_info(655386744))

UnboundLocalError: local variable 'birthday' referenced before assignment

In [43]:
user_info_df = pd.DataFrame()
user_info_df['id_пользователя'] = users_data['id_пользователя'].unique()
user_info_df['Инфо'] = user_info_df['id_пользователя'].apply(user_info)

user_info_df['Имя'] = user_info_df['Инфо'].apply(lambda x: x[0])
user_info_df['Фамилия'] = user_info_df['Инфо'].apply(lambda x: x[1])
user_info_df['Дата_рождения'] = user_info_df['Инфо'].apply(lambda x: x[2])
user_info_df['Закрытость профиля'] = user_info_df['Инфо'].apply(lambda x: x[3])
user_info_df['Город'] = user_info_df['Инфо'].apply(lambda x: x[4])

user_info_df = user_info_df.drop(['Инфо'], axis=1) 

user_info_df

UnboundLocalError: local variable 'birthday' referenced before assignment

,id_пользователя,инфо,Имя,Фамилия,Город
0,655386741,"[Kvantovy, Skachok, 0]",Kvantovy,Skachok,0
1,282596539,"[Svitlana, Smyshlyaeva, Kirov]",Svitlana,Smyshlyaeva,Kirov
2,69696727,"[Dmitry, Pestryakov, Saint Petersburg]",Dmitry,Pestryakov,Saint Petersburg
3,241770145,"[Stanislav, Βinogradov, Cherkassy]",Stanislav,Βinogradov,Cherkassy
4,426871262,"[Omich, Korennoy, 0]",Omich,Korennoy,0
...,...,...,...,...,...
762,277447170,"[Rus, Pirkuliev, 0]",Rus,Pirkuliev,0
763,6674901,"[Sid, Vashingtonovich, Ufa]",Sid,Vashingtonovich,Ufa
764,578975362,"[Ildar, Isaev, 0]",Ildar,Isaev,0
765,733345539,"[Vitaly, Babkin, 0]",Vitaly,Babkin,0


In [ ]:
users_data['Инфо'] = users_data['id_пользователя'].apply(user_info)

In [ ]:
users_data['Имя'] = users_data['Инфо'].apply(lambda x: x[0])
users_data['Фамилия'] = users_data['Инфо'].apply(lambda x: x[1])
users_data['Город'] = users_data['Инфо'].apply(lambda x: x[2])

users_data = users_data.drop(['Инфо'], axis=1) 

In [ ]:
users_data.head(10)

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город
0,-63683472,2794790,266884711,"Да пхд если бк детонирует, особо разницы нету,...",Pavel,Khadanov,Ulan-Ude
1,-63683472,2794790,9123584,Тупейший пост. Башня танка наиболее уязвимая ч...,Sergey,Alexandrovich,Lobnya
2,-63683472,2794790,317899,"Да если рванет, уже пох будет, во всех вариант...",Svyatoslav,Strizh,Gatchina
3,-63683472,2794790,40698122,,Kirill,Andreev,0
4,-63683472,2794790,405251797,"Лучше всего отношение к людям, стране показыва...",Elena,Lanskaya,Moscow
5,-63683472,2794790,139452093,А не система автодосылания и автозаряда?,Valery,Shvetsov,0
6,-63683472,2794790,344826496,"[id9123584|Сергей], При всем при этом экпижа А...",Aisberg,Zelov,Kraków
7,-63683472,2794790,635436564,Меня всё чаще начинают терзать сомнения.... Ад...,Mark,Kaprizov,Kozelsk
8,-63683472,2794790,442457832,"[id405251797|Елена], в Австралии война была в ...",Mixns,Mixns,0
9,-63683472,2794790,151564274,"""Солдатиков новых забрееют!!! А вот где я хоро...",Khamlo,Chulpanov,0


In [ ]:
# Генерируем ссылку на пост с комментариями
def url_to_post(id_group, id_post):
    return f"https://vk.com/atheism?w=wall{id_group}_{id_post}"

users_data['Ссылка_на_пост'] = 0
for i in range(users_data.shape[0]):
    id_group = users_data['id_группы'].iloc[i]
    id_post = users_data['id_поста'].iloc[i]
    users_data['Ссылка_на_пост'].iloc[i] = url_to_post(id_group, id_post)

c:\Users\mish9\.conda\envs\sf\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Генерируем ссылку на аккаунт пользователя
def url_to_user(id_user):
    return f"https://vk.com/id{id_user}"

users_data['Ссылка_на_пользователя'] = users_data['id_пользователя'].apply(url_to_user)

In [ ]:
users_data

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город,Ссылка_на_пост,Ссылка_на_пользователя
0,-63683472,2794790,266884711,"Да пхд если бк детонирует, особо разницы нету,...",Pavel,Khadanov,Ulan-Ude,https://vk.com/atheism?w=wall-63683472_2794790,https://vk.com/id266884711
1,-63683472,2794790,9123584,Тупейший пост. Башня танка наиболее уязвимая ч...,Sergey,Alexandrovich,Lobnya,https://vk.com/atheism?w=wall-63683472_2794790,https://vk.com/id9123584
2,-63683472,2794790,317899,"Да если рванет, уже пох будет, во всех вариант...",Svyatoslav,Strizh,Gatchina,https://vk.com/atheism?w=wall-63683472_2794790,https://vk.com/id317899
3,-63683472,2794790,40698122,,Kirill,Andreev,0,https://vk.com/atheism?w=wall-63683472_2794790,https://vk.com/id40698122
4,-63683472,2794790,405251797,"Лучше всего отношение к людям, стране показыва...",Elena,Lanskaya,Moscow,https://vk.com/atheism?w=wall-63683472_2794790,https://vk.com/id405251797
...,...,...,...,...,...,...,...,...,...
3399,-63683472,2788625,438982077,"[id27199103|Наталья], а вы уже сделали😉",Marina,Gornostaeva,Livny,https://vk.com/atheism?w=wall-63683472_2788625,https://vk.com/id438982077
3400,-63683472,2788625,4703517,"[id34457849|Михаил], а что хорошего в этой нов...",Sergey,Rusakov,0,https://vk.com/atheism?w=wall-63683472_2788625,https://vk.com/id4703517
3401,-63683472,2788625,356789277,АХАХАХХА,Blue,Sea,Saint Petersburg,https://vk.com/atheism?w=wall-63683472_2788625,https://vk.com/id356789277
3402,-63683472,2788625,69885373,"Да, да. А ещё https://youtu.be/elPxQsD8XoY",Koldun,Vudu,Vitebsk,https://vk.com/atheism?w=wall-63683472_2788625,https://vk.com/id69885373


# <center> Отфильтровываем данные с Иркутском

In [ ]:
Irkutsk = users_data[users_data['Город']=='Irkutsk']
Irkutsk

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Город,Ссылка_на_пост,Ссылка_на_пользователя
207,-63683472,2794126,33196285,"Видите, там даже бомжи более культурные и прия...",Lyonya,Golubkov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2794126,https://vk.com/id33196285
1839,-63683472,2791046,342957915,А ладу дадут?😆,Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2791046,https://vk.com/id342957915
2018,-63683472,2790974,342957915,"[id34457849|Михаил], это цивилизованные методы...",Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790974,https://vk.com/id342957915
2299,-63683472,2790636,171716341,"Ну педераст, товарищи, пе-де-раст... Что нового?)",Boris,Pavlov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790636,https://vk.com/id171716341
2342,-63683472,2790229,342957915,"Главное не гейевропеец, а то скрепы да и сами ...",Ksenia,Petrovna,Irkutsk,https://vk.com/atheism?w=wall-63683472_2790229,https://vk.com/id342957915
2830,-63683472,2789278,12960639,Интересно а как будут служить 30 летние с 18 л...,Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639
2861,-63683472,2789278,12960639,"[id669233873|Ханк], научи как читать жопой?",Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639
2862,-63683472,2789278,12960639,Согласен,Vovchik,Zakharov,Irkutsk,https://vk.com/atheism?w=wall-63683472_2789278,https://vk.com/id12960639


In [ ]:
users_data.to_csv('users_data.csv', sep=';', decimal='.', index=False)